In [ ]:
from google.colab import drive
drive.mount('/content/drive')-

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 -q
!pip install yolov5 -q
!pip install datasets -q
!pip install --upgrade diffusers transformers accelerate datasets -q
!pip install accelerate transformers diffusers datasets safetensors -q
!pip install --upgrade diffusers -q
!pip install ultralytics -q


In [ ]:
!pip show diffusers

In [ ]:
# Đường dẫn đến tập dữ liệu
dataset_path = "/content/drive/MyDrive/txttoimage11/dataset"

    txttoimage11/
    ├──API tạo ảnh từ văn bản.ipynb # File code huấn luyện mô hình
    └── dataset/ # Thư mục chứa tập dữ liệu
          ├── naruto_1.jpg
          ├── naruto_1.txt
          ├──  naruto_2.jpg
          ├──  naruto_2.txt
          └── …

    """
    Phân tích hình ảnh trong thư mục, dự đoán nhãn bằng mô hình,
    và lưu nhãn vào file văn bản tương ứng.
    
    Args:
        dataset_path (str): Đường dẫn đến thư mục chứa ảnh.
        model (YOLO): Mô hình YOLO được huấn luyện để phân tích ảnh.
    """

In [ ]:
import os
from PIL import Image

def analyze_images(dataset_path, model):
    """
    Phân tích hình ảnh trong thư mục, dự đoán nhãn bằng mô hình,
    và lưu nhãn vào file văn bản tương ứng.
    """
    for filename in os.listdir(dataset_path):
        # Kiểm tra phần mở rộng của file
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(dataset_path, filename)
            label_path = os.path.splitext(image_path)[0] + ".txt"

            # Kiểm tra xem file nhãn đã tồn tại
            if not os.path.exists(label_path):
                try:
                    # Tải và phân tích ảnh
                    image = Image.open(image_path)
                    results = model(image)

                    # Trích xuất nhãn từ kết quả
                    labels = []
                    for *xyxy, conf, cls in results.xyxy[0]:  # Kết quả từ YOLO hoặc mô hình tương tự
                        label = model.names[int(cls)]  # Lấy tên nhãn từ ID lớp
                        labels.append(label)

                    # Lưu nhãn vào file
                    with open(label_path, "w") as f:
                        f.write(", ".join(labels))
                    print(f"Lưu nhãn: {label_path}")
                except Exception as e:
                    print(f"Lỗi xử lý {filename}: {e}")
            else:
                print(f"File nhãn đã tồn tại: {label_path}")

# Ví dụ gọi hàm (cần thay 'dataset_path' và 'model' bằng dữ liệu của bạn)
# analyze_images("path/to/your/dataset", your_model_instance)


In [ ]:
from ultralytics import YOLO

# Tải mô hình YOLOv5
model = YOLO("yolov5s.pt")  # Thay bằng mô hình YOLO khác nếu cần

# Gọi hàm phân tích ảnh
analyze_images(dataset_path, model)


### Tiền xử lý ảnh: chuyển đổi ảnh PIL sang tensor và chuẩn hóa

In [ ]:
def preprocess(image):
    """
    Tiền xử lý ảnh: chuyển đổi ảnh PIL sang tensor và chuẩn hóa.
    """
    preprocess = transforms.Compose([
        transforms.Resize((512, 512)),  # Thay đổi kích thước ảnh
        transforms.ToTensor(),         # Chuyển ảnh PIL sang tensor
    ])
    image = preprocess(image)

    # Thêm một kênh vào tensor ảnh
    image = torch.cat([image, torch.ones_like(image[:1])], dim=0)

    return image

In [ ]:
# def preprocess(image):
#     """
#     Tiền xử lý ảnh: chuyển đổi ảnh PIL sang tensor và chuẩn hóa.
#     """
#     preprocess = transforms.Compose([
#         transforms.Resize((512, 512)),  # Thay đổi kích thước ảnh
#         transforms.ToTensor(),         # Chuyển ảnh PIL sang tensor
#         transforms.Normalize([0.5], [0.5])  # Chuẩn hóa dữ liệu (-1 đến 1)
#     ])
#     return preprocess(image)


# Thiết lập Dataset và DataLoader

In [ ]:
import os
import json
from PIL import Image
from torchvision import transforms
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch
from tqdm.auto import tqdm
from diffusers import StableDiffusionPipeline, DDPMScheduler
from torch.optim import AdamW
from accelerate import Accelerator
from flask import Flask, request, jsonify
import base64
from io import BytesIO

In [ ]:
# -*- coding: utf-8 -*-
import os
import json
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm.auto import tqdm
from diffusers import StableDiffusionPipeline, DDPMScheduler
from torch.optim import AdamW
from accelerate import Accelerator
from flask import Flask, request, jsonify
import base64
from io import BytesIO

class CustomDataset(Dataset):
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.image_files = [
            os.path.join(dataset_path, filename)
            for filename in os.listdir(dataset_path)
            if filename.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]  # Gán giá trị cho image_path
        image = Image.open(image_path).convert("RGB")
        image = preprocess(image)  # Di chuyển dòng này ra ngoài khối try...except

        label_path = os.path.splitext(image_path)[0] + ".txt"
        try:
            with open(label_path, "r") as f:
                label = f.read().strip().split(", ")
        except FileNotFoundError:
            print(f"Không tìm thấy tệp nhãn: {label_path}")
            label = []

        # Padding danh sách nhãn
        max_length = 10  # Giả sử độ dài tối đa của danh sách nhãn là 10
        label = label[:max_length]  # Cắt ngắn nếu danh sách dài hơn max_length
        label += ["<PAD>"] * (max_length - len(label))  # Thêm padding

        return {"pixel_values": image, "labels": label}

# Tạo DataLoader
train_dataset = CustomDataset(dataset_path)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)


In [ ]:
# def preprocess_function(examples):
#     images = [preprocess(image.convert("RGB")) for image in examples["image"]]
#     examples["pixel_values"] = images

#     # Trích xuất nhãn từ các tệp txt
#     labels = []
#     for i, image in enumerate(examples["image"]):
#         filename = examples["image_file_path"][i]
#         label_path = os.path.splitext(filename)[0] + ".txt"
#         try:
#             with open(label_path, "r") as f:
#                 # Đọc nội dung từ tệp txt và tách thành danh sách các nhãn
#                 label = f.read().strip().split(", ")
#             labels.append(label)
#         except FileNotFoundError:
#             print(f"Không tìm thấy tệp nhãn: {label_path}")
#             labels.append([])  # Hoặc xử lý lỗi theo cách khác

#     examples["labels"] = labels

#     return examples

# # Tải dataset bằng load_dataset
# dataset = load_dataset("imagefolder", data_dir=dataset_path)

# # Tạo danh sách files cho tập huấn luyện
# files = [os.path.join(dataset_path, filename) for filename in os.listdir(dataset_path) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))]

# # Thêm cột "image_file_path" cho tập huấn luyện
# def add_filepath(example, idx):
#     example['image_file_path'] = files[idx]  # Sử dụng danh sách files đã tạo
#     return example

# dataset['train'] = dataset['train'].map(add_filepath, with_indices=True)

# # Tiền xử lý dữ liệu TRƯỚC KHI tạo DataLoader
# dataset['train'] = dataset['train'].map(preprocess_function, batched=True, num_proc=4)

# # Tạo DataLoader
# train_dataloader = DataLoader(dataset["train"], batch_size=4, shuffle=True)


In [ ]:

# Tải mô hình Stable Diffusion
pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1")

# Sử dụng DPMSolverMultistepScheduler
pipe.scheduler = DDPMScheduler.from_config(pipe.scheduler.config)

# Chuyển model sang CPU
pipe.to("cpu")  # Thay đổi dòng này

# Optimizer
optimizer = AdamW(pipe.unet.parameters(), lr=5e-6)

# Số epochs
num_epochs = 10

In [ ]:
from transformers import CLIPTokenizer
# Khởi tạo tokenizer
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")


In [ ]:
import torch.nn.utils.rnn as rnn_utils

In [ ]:
# Gradient accumulation steps
gradient_accumulation_steps = 2

# Vòng lặp huấn luyện
for epoch in range(num_epochs):
    progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
    for step, batch in progress_bar:

        # Di chuyển dữ liệu sang CPU
        pixel_values = batch["pixel_values"].to("cpu")  # Thay đổi dòng này

        # Chuyển đổi batch["labels"] thành tensor sử dụng tokenizer
        labels = [torch.tensor(tokenizer.encode(l, add_special_tokens=False)) for l in batch["labels"]]
        labels = rnn_utils.pad_sequence(labels, batch_first=True, padding_value=tokenizer.pad_token_id).to("cpu")  # Thay đổi dòng này

        # Tạo noise ngẫu nhiên
        noise = torch.randn(pixel_values.shape).to("cpu")  # Thay đổi dòng này

        # Tạo timestep ngẫu nhiên
        timesteps = torch.randint(0, pipe.scheduler.config.num_train_timesteps, (pixel_values.shape[0],)).long().to("cpu")  # Thay đổi dòng này

        # Thêm noise vào ảnh gốc
        noisy_images = pipe.scheduler.add_noise(pixel_values, noise, timesteps)

        # Dự đoán noise
        encoder_hidden_states = pipe.text_encoder(labels)["last_hidden_state"]
        noise_pred = pipe.unet(noisy_images, timesteps, encoder_hidden_states=encoder_hidden_states).sample

        # Tính loss
        loss = torch.nn.functional.mse_loss(noise_pred, noise)

        # Backpropagation
        loss.backward()  # Thay đổi dòng này
        optimizer.step()
        optimizer.zero_grad()

        # Hiển thị loss trên progress bar
        progress_bar.set_postfix({"loss": loss.item()})


  0%|          | 0/21 [00:00<?, ?it/s]

# Lưu mô hình

In [ ]:
# Lưu mô hình
# pipe.save_pretrained("/content/drive/MyDrive/stable_diffusion_trained")

# Lưu mô hình sau khi huấn luyện
model.save_pretrained("/content/drive/MyDrive/txttoimage11")


chuyển đổi ảnh thành định dạng base64 trước khi trả về.

In [ ]:
import base64
from io import BytesIO

def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

@app.route('/generate', methods=['POST'])
def generate():
    description = request.form['description']
    prompt = f"{description}, no background"
    image = pipe(prompt).images[0]
    image_base64 = image_to_base64(image)
    return jsonify({'image': image_base64})


In [ ]:
from flask import Flask, request, jsonify
from diffusers import StableDiffusionPipeline

app = Flask(__name__)

# Tải mô hình Stable Diffusion đã huấn luyện
pipe = StableDiffusionPipeline.from_pretrained("path/to/save/model")
pipe = pipe.to("cuda")

@app.route('/generate', methods=['POST'])
def generate():
    description = request.form['description']
    # Xử lý mô tả và tạo prompt (ví dụ: thêm "no background")
    prompt = f"{description}, no background"
    image = pipe(prompt).images[0]
    # Chuyển đổi image sang base64 hoặc lưu vào file
    # ...
    return jsonify({'image': image_base64})  # Trả về ảnh

if __name__ == '__main__':
    app.run()